# Example 4 - Mapping Curie depth from EMAG2

Using global compilations of the magnetic anomaly, such as EMAG2, means there is no limitation on window sizes. The mapping module in *PyCurious* has several functions to simplify:

1. Translating between Coordinate Reference Systems (CRS)
2. Griding scattered point data
3. Importing and exporting Geotiffs

These can be accessed from `pycurious.mapping`

This requires some extra dependencies:

- pyproj
- pyepsg
- cartopy (for visualisation)

which can be installed via `pip`

```shell
pip install [--user] pyproj pyepsg cartopy
```

### Contents

- [Import EMAG2 data](#Import-EMAG2-data)
- [Interpolate onto grid projection](#Interpolate-onto-grid-projection)
- [Compute Curie depth](#Compute-Curie-depth)
- [Compare to reference models](#Compare-to-reference-models)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pycurious

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.shapereader as shpreader

## Import EMAG2 data

The latest data can be downloaded from the [NOAA web page](https://www.ngdc.noaa.gov/geomag/emag2.html) in CSV format. Download it to a local directory (may take some time).

Once loaded, we trim the data to a region of interest - being careful to allow some buffer to accommodate the maximum window size. We save this to a compressed `.npz` file to avoid the costly repitition of reloading the same data in the future.

In [ ]:
filedir = '/path/to/directory/'

try:
    with np.load(filedir+'EMAG2_V3_20170530.npz') as f:
        mag_data = f['data']
except:
    mag_data = np.loadtxt(filedir+'EMAG2_V3_20170530.csv', delimiter=',', usecols=(2,3,4,5,7))
    lon_mask = mag_data[:,0] > 180.0
    mag_data[lon_mask,0] -= 360.0
    np.savez_compressed(filedir+'EMAG2_V3_20170530.npz', data=mag_data.astype(np.float32))

In [ ]:
# filter NaNs
mag_data = mag_data[mag_data[:,4] != 99999.]
mag_data = mag_data[mag_data[:,4] != -888.]
mag_data = mag_data[mag_data[:,4] != -999.]

# print min/max
mincols = mag_data.min(axis=0)
maxcols = mag_data.max(axis=0)

fmt = "min/max {:5.2f} -> {:5.2f}"
for col in xrange(mag_data.shape[1]):
    print(fmt.format(mincols[col], maxcols[col]))

## Interpolate onto grid projection

We choose an arbitrary section of the Earth's surface and a suitable projection. PyCurious handles transformation between different CRS using a EPSG reference code. EMAG2 is in lons and lats (in decimal degrees), which we want to convert into eastings and northings (in metres). For this example, we transform WGS84 coordinates (EPSG:4326) to the IRENET95 projection (EPSG:2157).

In [ ]:
# grid extent - grid of the British Isles in Irenet 95 grid projection
xmin = 250000.0
xmax = 1400000.0
ymin = 200000.0
ymax = 1500000.0
extent_grid = [xmin, xmax, ymin, ymax]

# extent on the sphere (WGS84)
extent_sphere = pycurious.mapping.convert_extent(extent_grid, epsg_in=4326, epsg_out=2157)

# map extent - should be narrower than extent_sphere
extent_map = [-10.8, 2, 49.5, 59]

In [ ]:
dx, dy = 1e3, 1e3 # 1 km resolution
nx, ny = int(round((xmax-xmin)/dx)), int(round((ymax-ymin)/dy))

# interpolate onto grid extent (for computation)
mag_grid = pycurious.mapping.grid(mag_data[:,:2], mag_data[:,3], extent, shape=(ny,nx), epsg_in=4326, epsg_out=2157)

# interpolate onto extent on the sphere (for mapping)
mag_sphere = pycurious.mapping.grid(mag_data[:,:2], mag_data[:,3], extent_sphere, shape=(ny,nx))

In [ ]:
proj = ccrs.UTM(30)

fig = plt.figure(figsize=(12,10))
ax = plt.axes(projection=proj)
ax.set_extent(map_extent)
ax.coastlines(resolution='50m', linewidth=1.5)
ax.gridlines()

im1 = ax.imshow(mag_sphere, extent=extent_sphere, transform=ccrs.PlateCarree(),
                vmin=-200, vmax=200, zorder=0)

fig.colorbar(im1, label='nT')

## Compute Curie depth

For this example we use the Bouligand *et al.*, 2009 approach we outlined in [Ex3-Optimisation-routine](#./Ex3-Optimisation-routine.ipynb).

In [ ]:
grid = pycurious.CurieOptimise(mag_grid, xmin, xmax, ymin, ymax)

# centroid spacing of 20 km
xc_list, yc_list = grid.create_centroid_list(200e3, spacingX=20e3, spacingY=20e3)
print("number of centroids = {}".format(len(xc_list)))

beta, zt, dz, C = grid.optimise_routine(window_size, xc_list, yc_list)

## Compare to reference models

A global Curie depth reference model has been created by [Li *et al.*, 2017](https://www.nature.com/articles/srep45129), which is a useful resource to compare these results. Download the supplementary material from the open access [article](https://www.nature.com/articles/srep45129#supplementary-information) to the same directory as EMAG2.

In [ ]:
li_cpd = np.loadtxt(filedir + 'Li_2017_curiedepth.txt')
li_cpd = li_cpd[~np.isnan(li_cpd[:,2])]

li_grid = pycurious.grid(li_cpd[:,:2], li_cpd[:,2], extent_sphere, shape=(ny,nx))

In [ ]:
proj = ccrs.UTM(30)

fig = plt.figure(figsize=(12,10))
ax = plt.axes(projection=proj)
ax.set_extent(map_extent)
ax.coastlines(resolution='50m', linewidth=1.5)
ax.gridlines()

im1 = ax.imshow(li_grid, extent=extent_sphere, transform=ccrs.PlateCarree(),
                cmap='BrBG', zorder=0, vmin=10, vmax=40)

fig.colorbar(im1, label='km')